### TFX Components

This notebook shows how to create pipeline that uses TFX components:

* CsvExampleGen
* StatisticsGen
* SchemaGen
* ExampleValidator
* Transform
* Trainer
* Evaluator

In [ ]:
# Put your KFP cluster endpoint URL here if working from GCP notebooks (or local notebooks). ('https://xxxxx.notebooks.googleusercontent.com/')
kfp_endpoint='https://XXXXX.notebooks.googleusercontent.com/'

In [ ]:
input_data_uri = 'gs://ml-pipeline-playground/tensorflow-tfx-repo/tfx/components/testdata/external/csv'

#Only S3/GCS is supported for now.
module_file = 'gs://avolkov/tensorflow-tfx-repo/v0.29.0/tfx/examples/chicago_taxi_pipeline/taxi_utils.py'

In [ ]:
import kfp

In [ ]:
import json
from kfp.components import load_component_from_url

CsvExampleGen_op    = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/ExampleGen/CsvExampleGen/component.yaml')
StatisticsGen_op    = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/StatisticsGen/component.yaml')
SchemaGen_op        = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/SchemaGen/component.yaml')
ExampleValidator_op = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/ExampleValidator/component.yaml')
Transform_op        = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/Transform/component.yaml')
Trainer_op          = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/Trainer/component.yaml')
Evaluator_op        = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/9b0d47a226c61f96e1ebe7a8ba427df38f8734e1/components/deprecated/tfx/Evaluator/component.yaml')


def tfx_pipeline(
    input_data_uri,
):
    examples_task = CsvExampleGen_op(
        input_base=input_data_uri,
        input_config=json.dumps({
            "splits": [
                {'name': 'data', 'pattern': '*.csv'},
            ]
        }),
        output_config=json.dumps({
            "splitConfig": {
                "splits": [
                    {'name': 'train', 'hash_buckets': 2},
                    {'name': 'eval', 'hash_buckets': 1},
                ]
            }
        }),
    )
    
    statistics_task = StatisticsGen_op(
        examples=examples_task.outputs['examples'],
    )
    
    schema_task = SchemaGen_op(
        statistics=statistics_task.outputs['statistics'],
    )

    # Performs anomaly detection based on statistics and data schema.
    validator_task = ExampleValidator_op(
        statistics=statistics_task.outputs['statistics'],
        schema=schema_task.outputs['schema'],
    )

    # Performs transformations and feature engineering in training and serving.
    transform_task = Transform_op(
        examples=examples_task.outputs['examples'],
        schema=schema_task.outputs['schema'],
        module_file=module_file,
    )

    trainer_task = Trainer_op(
        module_file=module_file,
        examples=transform_task.outputs['transformed_examples'],
        schema=schema_task.outputs['schema'],
        transform_graph=transform_task.outputs['transform_graph'],
        train_args=json.dumps({'num_steps': 10000}),
        eval_args=json.dumps({'num_steps': 5000}),
    )

    # Uses TFMA to compute a evaluation statistics over features of a model and
    # perform quality validation of a candidate model (compared to a baseline).
    eval_config = {
        'model_specs': [{'signature_name': 'eval'}],
        'slicing_specs': [
            {},
            {'feature_keys': ['trip_start_hour']},
        ],
        'metrics_specs': [{
            'thresholds': {
                'accuracy': {
                    'value_threshold': {
                        'lower_bound': 0.6,
                    },
                    'change_threshold': {
                        'direction': 2,  # tfma.MetricDirection.HIGHER_IS_BETTER
                        'absolute': -1e-10,
                    }
                }
            }
        }],
    }
    model_analyzer = Evaluator_op(
        examples=examples_task.outputs['examples'],
        model=trainer_task.outputs['model'],
        eval_config=json.dumps(eval_config),
    )


kfp.Client(host=kfp_endpoint).create_run_from_pipeline_func(
    tfx_pipeline,
    arguments=dict(
        input_data_uri=input_data_uri,
    ),
)